Cleaning High Cardinality Data

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv")

Renaming Columns

In [3]:
df = df.rename(columns={
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "Salary",
    "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "AdditionalComp",
    "Please indicate the currency": "Currency",
    "How old are you?": "Age",
    "What industry do you work in?": "Industry",
    "Job title": "JobTitle",
    "What country do you work in?": "Country",
    "How many years of professional work experience do you have overall?": "ExperienceOverall",
    "How many years of professional work experience do you have in your field?": "ExperienceField",
    "What is your highest level of education completed?": "Education",
    "What is your gender?": "Gender",
    "What is your race? (Choose all that apply.)": "Race"
})

Changing ',' separation for thousands to float

In [4]:
df["Salary"] = df["Salary"].str.replace(",", "").astype(float)
df["AdditionalComp"] = df["AdditionalComp"].astype(str).str.replace(",", "").replace("nan", None)
df["AdditionalComp"] = pd.to_numeric(df["AdditionalComp"], errors="coerce")

Converting Currency

In [5]:
currency_rates = {
    'USD': 1.0,
    'GBP': 1.25,
    'EUR': 1.08,
    'CAD': 0.74,
    'AUD': 0.66,
    'INR': 0.012,
    'JPY': 0.0065,
    'SGD': 0.74,
    'MYR': 0.21,
    'IDR': 0.000065,
    
}

def convert_to_usd(row):
    rate = currency_rates.get(row['Currency'], None)
    if rate is not None:
        row['Salary'] = row['Salary'] * rate
        if not pd.isna(row['AdditionalComp']):
            row['AdditionalComp'] = row['AdditionalComp'] * rate
        row['Currency'] = 'USD'
    return row

df = df.apply(convert_to_usd, axis=1)

Handling Free-Text Country Names

In [6]:
df['Country_cleaned'] = df['Country'].str.strip().str.lower()

1. Manual Normalization

In [7]:
import re

# Sample array (full array already provided)
unique_locations = df["Country"].unique()


replacements = {
    'United States': [
        r'\bU\.?S\.?A?\.?\b', r'\bU\.S\.A\.?\b', r'\bU\.S\.?\.?\b',
        r'\bUsa\b', r'\bus\b', r'\bUS\b', r'\bUnited States\b',
        r'\bUnited states\b', r'\bUnited states of america\b',
        r'\bUnited States of America\b', r'\bAmerica\b',
        r'\bUnited Stated\b', r'\bUnited Statws\b', r'\bUnited Statues\b',
        r'\bUnited Stattes\b', r'\bUnited Statea\b', r'\bUSA\b',
        r'\busa\b', r'\bUNITED STATES\b', r'\bunited states\b',
        r'\bUnitedStates\b', r'\bUnited STates\b', r'\bUntied States\b',
        r'\bUnites States\b', r'\bUnited State\b', r'\bUnited Sates\b',
        r'\bUnite States\b', r'\bUnitef Stated\b', r'\bUnited Statss\b',
        r'\bUnited States is America\b', r'\bUS of A\b', r'\bUnited y\b',
        r'\bUSAB\b', r'\bUnited Stares\b', r'\bUSA tomorrow\b',
        r'\bU\. S\.?\b', r'\bU\.A\.\b', r'\bU\.S>\b', r'\bUSS\b',
        r'\bUSA-- Virgin Islands\b', r'\bUSaa\b', r'\buSA\b',
        r'\bUsa \b', r'\bUnited States of American\b',
        r'\bUnited States Of America\b'
    ],
    'United Kingdom': [
        r'\bUK\b', r'\buk\b', r'\bUk\b', r'\bU\.K\.?\b',
        r'\bUnited Kingdom\b', r'\bEngland\b', r'\bGreat Britain\b',
        r'\bScotland\b', r'\bWales\b', r'\bEngland/UK\b',
        r'\bUnited Kindom\b', r'\bUnited Kingdomk\b', r'\bU\.K. \(northern England\)\b',
        r'\bEngland, UK\b', r'\bEngland, United Kingdom\b',
        r'\bEngland, United Kingdom \b', r'\bEngland, Gb\b',
        r'\bNorthern Ireland\b', r'\bUK \(Northern Ireland\)\b',
        r'\bUnited kingdom\b', r'\bUnited kingdom \b', r'\bunited kingdom\b',
        r'\bScotland, UK\b', r'\bWales \(United Kingdom\)\b',
        r'\bUK, remote\b', r'\bUK, but for globally fully remote company\b',
        r'\bUK \(England\)\b'
    ],
    'Canada': [
        r'\bCanada\b', r'\bcanada\b', r'\bCanada, Ottawa, ontario\b',
        r'\bCanada and USA\b', r'\bCANADA\b', r'\bCANADA \b',
        r'\bCanadw\b', r'\bCsnada\b', r'\bCanad\b', r'\bCanadá\b'
    ],
    'Australia': [
        r'\bAustralia\b', r'\bAustralia \b', r'\baustralia\b',
        r'\bAustrali\b', r'\bAustralian \b'
    ],
    'Netherlands': [
        r'\bThe Netherlands\b', r'\bthe Netherlands\b', r'\bNetherlands\b',
        r'\bnetherlands\b', r'\bNetherlands \b', r'\bThe Netherlands \b',
        r'\bNederland\b', r'\bNL\b'
    ],
    'Germany': [
        r'\bGermany\b', r'\bGermany \b', r'\bgermany\b'
    ],
    'India': [
        r'\bIndia\b', r'\bindia\b', r'\bINDIA\b', r'\bibdia\b'
    ],
    'France': [
        r'\bFrance\b', r'\bFRANCE\b', r'\bfrance\b', r'\bFrance \b'
    ],
    'Mexico': [
        r'\bMexico\b', r'\bMexico \b', r'\bMéxico\b'
    ],
    'New Zealand': [
        r'\bNew Zealand\b', r'\bNew zealand\b', r'\bNZ\b',
        r'\bNew Zealand Aotearoa\b', r'\bAotearoa New Zealand\b',
        r'\bFrom New Zealand but on projects across APAC\b'
    ],
    'Singapore': [
        r'\bSingapore\b', r'\bsingapore\b', r'\bSingapore \b'
    ],
    'HongKong':[
        r'\bHong KongKong\b'
    ]
}

# Function to normalize location
def normalize_location(loc):
    for standard, patterns in replacements.items():
        for pattern in patterns:
            if re.search(pattern, loc, re.IGNORECASE):
                return standard
    return loc.strip()

# Apply normalization
normalized_locations = [normalize_location(loc) for loc in unique_locations]
unique_normalized_locations = sorted(set(normalized_locations))

df['Country_cleaned'] = df['Country'].apply(normalize_location)


In [8]:
import re
from rapidfuzz import process, fuzz
import pycountry

2. Implementing Fuzzy Logic

In [ ]:
valid_countries = [country.name for country in pycountry.countries]
valid_countries += ['United States', 'United Kingdom'] 

def regex_normalize(loc):
    for standard, patterns in replacements.items():
        for pattern in patterns:
            if re.search(pattern, loc, re.IGNORECASE):
                return standard
    return None 

def fuzzy_match_country(loc, threshold=85):
    match = process.extractOne(loc, valid_countries, scorer=fuzz.ratio)
    if match and match[1] >= threshold:
        return match[0]
    return None

def normalize_location_full(loc):
    if not isinstance(loc, str):
        return None
    loc = loc.strip()
    
    norm = regex_normalize(loc)
    if norm:
        return norm
    
    return fuzzy_match_country(loc)

df['Country_cleaned'] = df['Country'].astype(str).apply(normalize_location_full)

df = df[df['Country_cleaned'].notnull()]


Generalize Industry Column

In [ ]:
df['Industry_cleaned'] = df['Industry'].str.strip().str.lower().fillna('')

def map_industry(industry):
    if any(x in industry for x in ['tech', 'software', 'it', 'developer', 'data', 'cyber']):
        return 'Tech'
    elif any(x in industry for x in ['finance', 'bank', 'accounting', 'investment', 'insurance']):
        return 'Finance'
    elif any(x in industry for x in ['health', 'hospital', 'medical', 'clinic', 'nurse', 'pharma']):
        return 'Healthcare'
    elif any(x in industry for x in ['education', 'school', 'university', 'teaching', 'academic', 'library', 'libraries','science', 'research']):
        return 'Education'
    elif any(x in industry for x in ['government', 'public administration', 'military']):
        return 'Government'
    elif any(x in industry for x in ['nonprofit', 'ngo', 'charity', 'social']):
        return 'Nonprofit'
    elif any(x in industry for x in ['retail', 'ecommerce', 'shopping', 'fashion']):
        return 'Retail'
    elif any(x in industry for x in ['media', 'entertainment', 'music', 'film','publishing']):
        return 'Media & Entertainment'
    elif any(x in industry for x in ['law', 'legal', 'attorney']):
        return 'Legal'
    elif any(x in industry for x in ['construction', 'real estate', 'architecture']):
        return 'Construction & Real Estate'
    elif any(x in industry for x in ['consulting', 'strategy', 'business']):
        return 'Consulting'
    elif any(x in industry for x in ['manufacturing', 'engineering', 'mechanical']):
        return 'Engineering & Manufacturing'
    elif any(x in industry for x in ['transport', 'logistics', 'supply chain']):
        return 'Logistics'
    elif any(x in industry for x in ['marketing','advertising','sales']):
        return 'Marketing'
    elif any(x in industry for x in ['art', 'design']):
        return 'Art & Design'
    elif any(x in industry for x in ['agriculture', 'forestry']):
        return 'Agriculture'
    
    else:
        return 'Other'

df['Industry_grouped'] = df['Industry_cleaned'].apply(map_industry)

print(df['Industry_grouped'].value_counts())

Industry_grouped
Tech                           9390
Education                      3764
Finance                        2351
Healthcare                     2099
Government                     1917
Engineering & Manufacturing    1768
Marketing                      1426
Legal                          1097
Other                           944
Consulting                      891
Retail                          515
Construction & Real Estate      450
Art & Design                    370
Media & Entertainment           337
Logistics                       315
Nonprofit                       274
Agriculture                     138
Name: count, dtype: int64


Generalize JobTitle Column

In [ ]:
df['JobTitle_cleaned'] = df['JobTitle'].astype(str).str.strip().str.lower().fillna('')

In [ ]:
def map_job_title(title):
    title = str(title).lower()
    if any(x in title for x in ['ceo', 'cfo','cto','director','chief', 'executive', 'president', 'vp', 'vice','head']):
        return 'Executive'
    elif any(x in title for x in ['coordinator','manager', 'supervisor', 'lead','senior']):
        return 'Management'
    elif any(x in title for x in ['computer','agile','scrum','engineer', 'developer', 'programmer', 'software', 'tech', 'technician', 'it']):
        return 'Engineering & IT'
    elif any(x in title for x in ['data', 'analyst', 'scientist', 'analytics','statistician']):
        return 'Data & Analytics'
    elif any(x in title for x in ['designer','artist','graphic','design']):
        return 'Creative Arts'
    elif any(x in title for x in ['school','teaching','Bookkeeper','lecturer','library','librarian','teacher', 'instructor', 'educator', 'professor', 'tutor', 'trainer', 'instructional','education','research']):
        return 'Education & Training'
    elif any(x in title for x in ['outreach','pr','marketing', 'brand', 'communications', 'content', 'seo','reporter']):
        return 'Marketing & Communications'
    elif any(x in title for x in ['sales', 'account executive', 'business development']):
        return 'Sales'
    elif any(x in title for x in ['hr', 'human resources', 'recruiter', 'talent']):
        return 'Human Resources'
    elif any(x in title for x in ['archivist','admin', 'administrative', 'office', 'clerk','associate']):
        return 'Administrative'
    elif any(x in title for x in ['risk','payroll','finance', 'accounting', 'accountant', 'cpa', 'auditor','economist','buyer']):
        return 'Finance & Accounting'
    elif any(x in title for x in ['legal', 'lawyer', 'attorney', 'paralegal','compliance','advocate']):
        return 'Legal'
    elif any(x in title for x in ['dental','clinician','surgeon','clinical','psychiatrist','veterinary','optometrist','veterinarian','pharmacist','nurse', 'doctor', 'physician', 'medical', 'health','therapist']):
        return 'Healthcare'
    elif any(x in title for x in ['counsel','consultant', 'consulting', 'advisor','psychologist','adviser']):
        return 'Consulting'
    elif any(x in title for x in ['customer service', 'support', 'help desk']):
        return 'Customer Service'
    elif any(x in title for x in ['producer','project', 'program','partner','assistant']):
        return 'Project & Program Management'
    elif any(x in title for x in ['owner']):
        return 'Entrepreneur'
    elif any(x in title for x in ['biologist', 'epidemiologist','geologist','archaeologist','biostatistician','chemist','astronomer','sciences','ecologist','scientific']):
        return 'Researcher & Scientist'
    elif any(x in title for x in ['journalist','reporter','media','author']):
        return 'News & Media'
    elif any(x in title for x in ['translator']):
        return 'Language Expert'
    elif any(x in title for x in ['assistant','controller','worker']):
        return 'Assistant'
    elif any(x in title for x in ['intern','student']):
        return 'Intern'
    elif any(x in title for x in ['planner','strategist','planning','investigator','predictor','specialist','tester']):
        return 'Strategist'
    else:
        return 'Other'
df['JobTitle_grouped'] = df['JobTitle_cleaned'].apply(map_job_title)

df['JobTitle_grouped'].value_counts()


JobTitle_grouped
Management                      9216
Executive                       4490
Engineering & IT                4103
Education & Training            1620
Data & Analytics                1606
Administrative                  1220
Other                           1146
Marketing & Communications       992
Legal                            514
Consulting                       512
Strategist                       502
Healthcare                       440
Creative Arts                    331
Finance & Accounting             282
Human Resources                  265
Project & Program Management     247
Assistant                        140
Sales                            107
Researcher & Scientist           100
Intern                            71
Customer Service                  67
News & Media                      42
Entrepreneur                      17
Language Expert                   16
Name: count, dtype: int64

Selecting Clean Columns for Step 1

In [18]:
df = df[['Age','Industry_grouped','JobTitle_grouped','Salary','AdditionalComp','Country_cleaned','ExperienceOverall','ExperienceField','Education','Gender','Race']]

In [21]:
df.to_csv("CleanedDataset.csv", index=False)

CleanedDataset will be the one used 